In [2]:
import numpy as np

In [3]:
cd kaggle

/Users/martiom/kaggle


In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import preprocessing as pre
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression


In [3]:
g = GBC()

In [4]:
g.fit?

In [19]:
train = pd.read_csv('train.csv')
train.replace([' ', '', -1], np.nan, inplace = True)
train['NaNCount'] = train.isnull().sum(axis=1)

In [20]:
def convert_date_data(date_data):
    date = pd.to_datetime(pd.Series(date_data))
    return np.vstack([date.dt.year, date.dt.month, 
                      date.dt.day, date.dt.dayofweek])

from collections import Counter
def extract_object_data(row):
    counts = Counter(row)
    row = row.fillna(max(filter(lambda x: x != np.nan, counts.keys()), 
               key = lambda x: counts[x]))
    data = pd.get_dummies(row).values
    if data.shape[1] == 1:
        print 'constant data'
        return []
    return data.T

def extract_numeric_data(row):
    row = row.fillna(row.median())
    if len(row.unique()) == 1:
        return []

    row = row - row.mean()
    row = row/row.std()
    return row.values

def extract_ordinal_data(row):
    return extract_numeric_data(row)

def convert_data(dataframe):
    y = dataframe.QuoteConversion_Flag.values
    X = []
    X.append(convert_date_data(dataframe['Original_Quote_Date']))
    columns_to_skip = ['Original_Quote_Date', 'QuoteConversion_Flag',
                      'QuoteNumber']
    for c in dataframe.columns:
        if c in columns_to_skip:
            print c, 'skipped'
            continue
        if train[c].dtype == object:
            X.append(extract_object_data(dataframe[c]))
        elif c.endswith('A') or c.endswith('B'):
            X.append(extract_ordinal_data(dataframe[c]))
        else:
            X.append(extract_numeric_data(dataframe[c]))
    X = filter(lambda x: len(x) > 0, X)
    return np.vstack(X).T, y


In [21]:
X, y = convert_data(train)

QuoteNumber skipped
Original_Quote_Date skipped
QuoteConversion_Flag skipped


In [22]:
np.save('train_X.npy', X)

In [23]:
np.save('train_y.npy', y)

In [13]:
from sklearn.linear_model import LogisticRegression


In [5]:
X, y = np.load('train_X.npy'), np.load('train_y.npy')

In [6]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.20, random_state=36)


In [8]:
def check_classifier(Xtrain, ytrain, Xtest, ytest, clf):
    clf.fit(Xtrain, ytrain)
    print clf.score(Xtrain, ytrain)
    print clf.score(Xtest, ytest)

In [26]:
check_classifier(Xtrain, ytrain, Xtest, ytest,LogisticRegression() )

0.916894373017
0.915552913655


In [27]:
clf = GBC()

In [29]:
check_classifier(Xtrain, ytrain, Xtest, ytest,clf )

0.922440820318
0.923088723131


In [16]:
%%timeit -r 1 -n 1
import xgboost as xgb
dc = lambda: xgb.XGBClassifier()

clf = dc()
check_classifier(Xtrain, ytrain, Xtest, ytest,clf )

clf = dc()
clfbag = BaggingClassifier(clf, n_estimators=5)
check_classifier(Xtrain, ytrain, Xtest, ytest,clfbag )

clf = dc()
clf_isotonic = CalibratedClassifierCV(clf, cv=5, method='isotonic')
check_classifier(Xtrain, ytrain, Xtest, ytest,clf_isotonic )

0.921606696005
0.921688941727
0.921692984727
0.921631416464
0.923495460254
0.924066652605


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

dc = lambda: KNeighborsClassifier(3)

clf = dc()
check_classifier(Xtrain, ytrain, Xtest, ytest,clf )

clf = dc()
clfbag = BaggingClassifier(clf, n_estimators=5)
check_classifier(Xtrain, ytrain, Xtest, ytest,clfbag )

clf = dc()
clf_isotonic = CalibratedClassifierCV(clf, cv=5, method='isotonic')
check_classifier(Xtrain, ytrain, Xtest, ytest,clf_isotonic )

In [9]:
clf = GBC()
clfbag = BaggingClassifier(clf, n_estimators=5)
check_classifier(Xtrain, ytrain, Xtest, ytest,clfbag )

clf = GBC()
clf_isotonic = CalibratedClassifierCV(clf, cv=5, method='isotonic')
check_classifier(Xtrain, ytrain, Xtest, ytest,clf_isotonic )

KeyboardInterrupt: 